In [ ]:
from torch.utils.data  import DataLoader

In [ ]:
import torch

In [ ]:
t = torch.arange(6, dtype=torch.float32)

In [ ]:
da = DataLoader(t)
for item in da:
  print(item)

tensor([0.])
tensor([1.])
tensor([2.])
tensor([3.])
tensor([4.])
tensor([5.])


In [ ]:
data = DataLoader(t, batch_size=2, drop_last=False)
val = [value.numpy() for value in data]
print(val)

[array([0., 1.], dtype=float32), array([2., 3.], dtype=float32), array([4., 5.], dtype=float32)]


In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

In [13]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
X = iris['data']
y = iris['target']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=1./3)

In [17]:
import numpy as np
X_train_norm = (X_train - np.mean(X_train))/np.std(X_train)

In [18]:
X_train_norm = torch.from_numpy(X_train_norm).float()

In [19]:
y_train_norm = torch.from_numpy(y_train)

In [21]:
from torch.utils.data import TensorDataset

In [22]:
train_ds = TensorDataset(X_train_norm, y_train_norm)

In [23]:
from torch.utils.data import DataLoader

In [24]:
torch.manual_seed(1)
batch_size =2
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [25]:
import torch.nn as nn
class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x = self.layer1(x)
    x = nn.Sigmoid()(x)
    x = self.layer2(x)
    x = nn.Softmax(dim=1)(x)
    return x

In [26]:
input_size = X_train_norm.shape[1]
hidden_size = 16
output_size = 3
model = Model(input_size, hidden_size, output_size)

In [27]:
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr =learning_rate)

In [33]:
num_epochs = 100
log_epochs = 10
for epoch in range(num_epochs):
  for x_batch, y_batch in train_dl:
    pred = model(x_batch)
    loss = loss_fn(pred, y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  if epoch % log_epochs == 0:
    print(f'Epoch {epoch} Loss {loss.item():.4f}')

Epoch 0 Loss 0.6028
Epoch 10 Loss 0.5519
Epoch 20 Loss 0.8049
Epoch 30 Loss 0.5515
Epoch 40 Loss 0.5649
Epoch 50 Loss 0.5543
Epoch 60 Loss 0.7585
Epoch 70 Loss 0.5517
Epoch 80 Loss 0.5515
Epoch 90 Loss 0.5576


In [34]:
X_test_norm = (X_test - np.mean(X_train))/np.std(X_train)
X_test_norm = torch.from_numpy(X_test_norm).float()
y_test_norm = torch.from_numpy(y_test)

In [35]:
pred_test = model(X_test_norm)

In [36]:
correct = (torch.argmax(pred_test, dim=1) == y_test).float()

In [37]:
accuracy = correct.mean()

In [38]:
print(f'Test Acc.: {accuracy:.4f}')

Test Acc.: 1.0000


In [40]:
path = 'iris_classifier.pt'
torch.save(model.state_dict(), path)

In [41]:
model_new = Model(input_size, hidden_size, output_size)
model_new.load_state_dict(torch.load(path))

<ipython-input-41-b21dbd0d4824>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_new.load_state_dict(torch.load(path))


<All keys matched successfully>